# Preparing the receptor

In [3]:
from openeye import oechem, oedocking
import sys, os

In [5]:
ifs = oechem.oemolistream()
ofs = oechem.oemolostream()

In [6]:
com = oechem.OEGraphMol()
if ifs.open("input/com_AXI.pdb"):
    oechem.OEReadPDBFile(ifs,com)
    ifs.close()

In [10]:
# Find the connected components of the system, remove the ligand
oechem.OEDetermineConnectivity(com)
connect = oechem.OEDetermineComponents(com)

print('Number of connected components:', connect[0])
if connect[0] != 1:
    # We assume that the last object is the ligand
    res_ind = [i for i,x in enumerate(connect[1]) if x==connect[0]]

Number of connected components: 2


This will break if the last connected component isn't the ligand
I don't know what would happen if the protein has multiple chains.


In [11]:
# Now iterate over these indices to create a box for docking
x_min = y_min = z_min = float('inf')
x_max = y_max = z_max = -float('inf')
crd = [com.GetCoords()[i] for i in res_ind]
for x,y,z in crd:
    if x < x_min:
        x_min = x
    if y < y_min:
        y_min = y
    if z < z_min:
        z_min = z
    if x > x_max:
        x_max = x
    if y > y_max:
        y_max = y
    if z > z_max:
        z_max = z

In [ ]:
# Pad the box to be slightly larger than the existing molecule
# My choice of padding by 2 Angstrom is arbitrary.
x_min -= 2
y_min -= 2
z_min -= 2
x_max += 2
y_max += 2
z_max += 2

In [ ]:
# Now prepare the receptor
receptor = oechem.OEGraphMol()
box = oedocking.OEBox(x_max, y_max, z_max, x_min, y_min, z_min)
oedocking.OEMakeReceptor(receptor, com, box)

In [ ]:
# Clean the receptor as needed
if oedocking.OEReceptorHasBoundLigand(receptor):
    lig = oedocking.OEReceptorGetBoundLigand(receptor)
    ofs = oechem.oemolostream("original_lig.mol2")
    oechem.OEWriteMolecule(ofs,lig)
    ofs.close()
    oedocking.OEReceptorClearBoundLigand(receptor)

In [ ]:
oedocking.OEWriteReceptorFile(receptor,"input/receptor.oeb")

Need to clean the receptor.
Right now it's not recognizing the ligand, neither as a ligand nor as an extra molecule